In [65]:
import numpy as np
import pdfplumber
import os
import re
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Border, Alignment
from pathlib import Path

### Read Does Parameters TXT file and Convert A(Bq) to uSv

In [82]:
def CalDoes(excel_path, doespar_path):
    df = pd.read_excel(excel_path, sheet_name='Summary_Act')
    doespar_df = pd.read_csv(doespar_path, sep='\\s+', names=['A', 'Isotope', 'DoesPar'], comment='#')

    # Filter columns containing "cd"
    cd_columns = [col for col in df.columns if "cd" in col]

    for idx, row in df.iterrows():
        # Get the current "A" and "Isotope" in the Excel sheet
        A = int(row['A'])
        isotope = row['isotope']
        match = doespar_df[(doespar_df['A'].astype(int) == A) & (doespar_df['Isotope'] == isotope)]
        if not match.empty:
            # Get the DoesPar value from the txt file
            does_par = match['DoesPar'].values[0]
            # Multiply DoesPar with each of the cd columns
            for col in cd_columns:
                new_col_name = f"Does(uSv)@{col[6:]}"  # Create new column name by adding Does(uSv)
                df.loc[idx, new_col_name] = row[col]/1e9 * does_par  # Calculate and assign new value
        else:
            df = df.drop(idx)

    # Sum all "Does(uSv)" columns and add a new row with the total sum
    does_columns = [col for col in df.columns if col.startswith("Does(uSv)")]  # Find all the new "Does(uSv)" columns
    sum_row = df[does_columns].sum()  # Calculate the sum of all the new "Does(uSv)" columns

    # Create a sum row and add it to the DataFrame
    sum_row_label = ['Total'] + [''] * (df.shape[1] - len(does_columns) - 1)  # Set 'Total' label for first column
    sum_row_full = pd.DataFrame([sum_row_label + sum_row.tolist()], columns=df.columns)

    # Append the sum row to the DataFrame
    df = pd.concat([df, sum_row_full], ignore_index=True)
    doespar_file = Path(doespar_path)
    sheet_name = f"{doespar_file.stem}"  # Create the new sheet name

    with pd.ExcelWriter(excel_path, mode='a', engine='openpyxl', if_sheet_exists = 'replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook[sheet_name]
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 14
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
            # Save the workbook
    workbook.save(excel_path)

In [83]:
excel_path = "example/DoesCal.xlsx"
txt_file_path1 = "example/DoesPar/Body_unshield.txt"
txt_file_path2 = "example/DoesPar/Body_shield.txt"
CalDoes(excel_path, txt_file_path1) 
CalDoes(excel_path, txt_file_path2) 